In [1]:
import pickle
import glob

dirname = './noisy_benchmark_data'
files = glob.glob('%s/evaluator_input*.p'%dirname)
for f in files:
    dimension,searcher_time,circ,fc_evaluations,clusters,complete_path_map = pickle.load(open(f, 'rb' ) )

    print(dimension)
    [print(x,complete_path_map[x]) for x in complete_path_map]

(2, 5)
(QuantumRegister(10, 'q2'), 0) [(0, 0)]
(QuantumRegister(10, 'q2'), 1) [(0, 1)]
(QuantumRegister(10, 'q2'), 2) [(0, 2), (1, 0), (2, 0)]
(QuantumRegister(10, 'q2'), 3) [(1, 1)]
(QuantumRegister(10, 'q2'), 4) [(1, 2)]
(QuantumRegister(10, 'q2'), 5) [(0, 3)]
(QuantumRegister(10, 'q2'), 6) [(0, 4)]
(QuantumRegister(10, 'q2'), 7) [(1, 3), (0, 5), (2, 1)]
(QuantumRegister(10, 'q2'), 8) [(1, 4)]
(QuantumRegister(10, 'q2'), 9) [(1, 5)]
(3, 3)
(QuantumRegister(9, 'q1'), 0) [(0, 0)]
(QuantumRegister(9, 'q1'), 1) [(0, 1), (1, 0)]
(QuantumRegister(9, 'q1'), 2) [(1, 1)]
(QuantumRegister(9, 'q1'), 3) [(0, 2)]
(QuantumRegister(9, 'q1'), 4) [(0, 3), (2, 0)]
(QuantumRegister(9, 'q1'), 5) [(2, 1), (1, 2)]
(QuantumRegister(9, 'q1'), 6) [(0, 4)]
(QuantumRegister(9, 'q1'), 7) [(0, 5), (3, 0)]
(QuantumRegister(9, 'q1'), 8) [(2, 2), (3, 1)]
(2, 4)
(QuantumRegister(8, 'q0'), 0) [(0, 0)]
(QuantumRegister(8, 'q0'), 1) [(0, 1), (1, 0)]
(QuantumRegister(8, 'q0'), 2) [(1, 1)]
(QuantumRegister(8, 'q0'), 3) [